In [ ]:
import tensorflow as tf
from tensorflow import keras
from keras import models
from keras.layers import Dense, Flatten, Dropout, Conv2D, MaxPooling2D, BatchNormalization, SeparableConv2D
import numpy as np
from keras.callbacks import EarlyStopping
from keras.preprocessing.image import ImageDataGenerator
from sklearn.model_selection import train_test_split
from keras.optimizers import Adam
from matplotlib import pyplot as plt

In [ ]:
fashion_mnist = keras.datasets.fashion_mnist

(train_images, train_labels), (test_images, test_labels) = fashion_mnist.load_data()

In [ ]:
print(train_images.shape, train_labels.shape)
print(test_images.shape, test_labels.shape)

(60000, 28, 28) (60000,)
(10000, 28, 28) (10000,)


In [ ]:
train_images = train_images.reshape(train_images.shape[0], 28, 28,1)
test_images = test_images.reshape(test_images.shape[0], 28, 28,1)

train_images = train_images.astype('float32') / 255.0
test_images = test_images.astype('float32') / 255.0

train_labels = keras.utils.to_categorical(train_labels, 10)
test_labels = keras.utils.to_categorical(test_labels, 10)

print(train_images.shape, test_images.shape)
print(test_images.shape, test_labels.shape)

(60000, 28, 28, 1) (10000, 28, 28, 1)
(10000, 28, 28, 1) (10000, 10)


In [ ]:
Train_images, val_images, Train_labels,  val_labels = train_test_split(train_images, train_labels, train_size = 0.8)

print('train :', Train_images.shape, Train_labels.shape)
print('val :', val_images.shape, val_labels.shape)
print('test :', test_images.shape, test_labels.shape)

train : (48000, 28, 28, 1) (48000, 10)
val : (12000, 28, 28, 1) (12000, 10)
test : (10000, 28, 28, 1) (10000, 10)


In [ ]:
augmentation_generator = ImageDataGenerator(
    rotation_range = 10, width_shift_range = 0.1, height_shift_range = 0.1, shear_range = 0.5, zoom_range = 0.3,
    horizontal_flip = True)

augmentation_generator.fit(Train_images)

In [ ]:
model = models.Sequential()

model.add(Conv2D(64, kernel_size=(3,3), activation = 'relu', padding = 'same', kernel_initializer= 'he_normal', input_shape = (28,28,1)))
model.add(BatchNormalization())
model.add(Conv2D(64, kernel_size=(3,3), activation = 'relu', padding = 'same'))
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size = (2,2)))
model.add(Dropout(0.35))

model.add(Conv2D(128, kernel_size=(3,3), activation = 'relu', padding = 'same', kernel_initializer= 'he_normal'))
model.add(BatchNormalization())
model.add(Conv2D(128, kernel_size=(3,3), activation = 'relu', padding = 'same'))
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size = (2,2)))
model.add(Dropout(0.35))

model.add(Conv2D(256, kernel_size=(3,3), activation = 'relu', padding = 'same', kernel_initializer= 'he_normal'))
model.add(BatchNormalization())
model.add(Conv2D(256, kernel_size=(3,3), activation = 'relu', padding = 'same'))
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size = (2,2)))
model.add(Dropout(0.35))

model.add(Flatten())

model.add(Dense(2048, activation = 'relu'))
model.add(Dropout(0.35))
model.add(Dense(512, activation = 'relu'))
model.add(Dense(10, activation = 'softmax'))

model.summary()


Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 28, 28, 64)        640       
_________________________________________________________________
batch_normalization (BatchNo (None, 28, 28, 64)        256       
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 28, 28, 64)        36928     
_________________________________________________________________
batch_normalization_1 (Batch (None, 28, 28, 64)        256       
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 14, 14, 64)        0         
_________________________________________________________________
dropout (Dropout)            (None, 14, 14, 64)        0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 14, 14, 128)       7

In [ ]:
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

In [ ]:
early_stopping = EarlyStopping(monitor='val_accuracy', patience=7, verbose=1)
model.fit_generator(augmentation_generator.flow(Train_images, Train_labels, batch_size= 30), epochs = 50, verbose=1, validation_data=(val_images, val_labels), callbacks= early_stopping)

/usr/local/lib/python3.7/dist-packages/keras/engine/training.py:1915: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  warnings.warn('`Model.fit_generator` is deprecated and '


Epoch 1/50
1600/1600 [==============================] - 71s 16ms/step - loss: 1.1760 - accuracy: 0.6309 - val_loss: 0.4683 - val_accuracy: 0.8162
Epoch 2/50
1600/1600 [==============================] - 23s 14ms/step - loss: 0.5758 - accuracy: 0.7883 - val_loss: 0.4751 - val_accuracy: 0.8247
Epoch 3/50
1600/1600 [==============================] - 23s 14ms/step - loss: 0.4966 - accuracy: 0.8188 - val_loss: 0.4381 - val_accuracy: 0.8396
Epoch 4/50
1600/1600 [==============================] - 23s 14ms/step - loss: 0.4503 - accuracy: 0.8365 - val_loss: 0.3751 - val_accuracy: 0.8658
Epoch 5/50
1600/1600 [==============================] - 23s 14ms/step - loss: 0.4067 - accuracy: 0.8543 - val_loss: 0.3587 - val_accuracy: 0.8712
Epoch 6/50
1600/1600 [==============================] - 23s 14ms/step - loss: 0.3919 - accuracy: 0.8585 - val_loss: 0.3458 - val_accuracy: 0.8734
Epoch 7/50
1600/1600 [==============================] - 23s 14ms/step - loss: 0.3810 - accuracy: 0.8626 - val_loss: 0.3476 -

In [ ]:
early_stopping = EarlyStopping(monitor='val_accuracy', patience=7, verbose=1)
model.fit(Train_images, Train_labels, validation_data=(val_images, val_labels), epochs=40, batch_size = 30, callbacks= early_stopping, verbose = 1)

prob_pred = model.predict(test_images)
prob_label = prob_pred.argmax(axis=-1)

np.savetxt('y_pred.csv', prob_label,fmt='%d')

Epoch 1/40
1600/1600 [==============================] - 20s 12ms/step - loss: 0.1647 - accuracy: 0.9406 - val_loss: 0.1600 - val_accuracy: 0.9444
Epoch 2/40
1600/1600 [==============================] - 18s 11ms/step - loss: 0.1491 - accuracy: 0.9460 - val_loss: 0.1502 - val_accuracy: 0.9470
Epoch 3/40
1600/1600 [==============================] - 18s 11ms/step - loss: 0.1387 - accuracy: 0.9490 - val_loss: 0.1563 - val_accuracy: 0.9461
Epoch 4/40
1600/1600 [==============================] - 17s 11ms/step - loss: 0.1302 - accuracy: 0.9531 - val_loss: 0.1701 - val_accuracy: 0.9389
Epoch 5/40
1600/1600 [==============================] - 17s 11ms/step - loss: 0.1188 - accuracy: 0.9565 - val_loss: 0.1706 - val_accuracy: 0.9442
Epoch 6/40
1600/1600 [==============================] - 17s 11ms/step - loss: 0.1144 - accuracy: 0.9576 - val_loss: 0.1654 - val_accuracy: 0.9457
Epoch 7/40
1600/1600 [==============================] - 17s 11ms/step - loss: 0.1073 - accuracy: 0.9610 - val_loss: 0.1638 -